## Backward Repeated A-Star maxG

In [17]:
import pygame
GREEN = (0, 255, 0) # EXPANDED
YELLOW = (255, 255, 0) # START
PATH = (0, 255, 255) # FINAL PATH 
RED = (255, 0, 0) # IN FRONTIER
BLUE = (0, 0, 255) #END
WHITE = (255, 255,255) #STANDARD
BLACK = (0,0,0) # BLOCKED
GREY = (127,127,127)#UNEXPANDED




ROWS = 101
NODE_LENGTH = 5

GRID_LENGTH = NODE_LENGTH*ROWS

START_NODE = (0,0)
END_NODE = (ROWS-1,ROWS-1)

def readFile(fname):
    fp = open(fname, 'r')
    maze = []
    for line in fp:
        row = []
        for block in line.split():
            if block.strip()=='1':
                row.append(BLACK)
            else:
                row.append(WHITE)
        maze.append(row)
    return maze

from asyncore import read
from msilib import OpenDatabase
from queue import PriorityQueue as PQ
import random
from tkinter import END
from tkinter.filedialog import Open
import pygame
from math import inf
import time
#from maze_gen_test import make_maze
#from maze_new_test import newmaze
successful = [2, 4, 6, 9, 12, 15, 17, 18, 20, 21, 22, 27, 28, 29, 30, 34, 39, 40, 44, 49, 50, 52, 54, 55, 61, 62, 65, 68, 69, 72, 77, 83, 85, 86, 88, 89, 97, 99]
unsuccessful = [1, 3, 5, 7, 8, 10, 11, 13, 14, 16, 19, 23, 24, 25, 26, 31, 32, 33, 35, 36, 37, 38, 41, 42, 43, 45, 46, 47, 48, 51, 53, 56, 57, 58, 59, 60, 63, 64, 66, 67, 70, 71, 73, 74, 75, 76, 78, 79, 80, 81, 82, 84, 87, 90, 91, 92, 93, 94, 95, 96, 98, 100]
# from reader import readFile
expanded_cells = []

actual_grid = [[0 for _ in range(ROWS)] for x in range(ROWS)]
agent_grid = [[GREY for _ in range(ROWS)] for x in range(ROWS)]
agent_grid[-1][-1] = BLUE
search = [[0 for _ in range(ROWS)] for x in range(ROWS)]
g = [[float('inf') for _ in range(ROWS)] for x in range(ROWS)]
f = [[float('inf') for _ in range(ROWS)] for x in range(ROWS)]
def calculate_hcost(ROWS, target):
    for i in range(ROWS):
        for j in range(ROWS):
            h[i][j] = abs(target[0]-i)+abs(target[1]-j)


def give_neighbors_update(NODE):
    retval = []
    i,j = NODE[0], NODE[1]
    if j-1>=0:
        retval.append((i, j-1))
    if j+1<ROWS:
        retval.append((i, j+1))
    if i-1>=0:
        retval.append((i-1, j))
    if i+1<ROWS:
        retval.append((i+1, j))
    return retval

def init_agent_grid():
    global agent_grid
    agent_grid = [[GREY for _ in range(ROWS)] for x in range(ROWS)]
    agent_grid[START_NODE[0]][START_NODE[1]] = RED
    agent_grid[END_NODE[0]][END_NODE[1]] = BLUE
    neighbors = give_neighbors_update(START_NODE)
    for neighbor in neighbors:
        agent_grid[neighbor[0]][neighbor[1]] = actual_grid[neighbor[0]][neighbor[1]]
    #update_frame()

def set_start(i,j):
    global START_NODE
    actual_grid[START_NODE[0]][START_NODE[1]] = WHITE #Make old start white
    #display_grid[i][j] = RED #color the new start node
    agent_grid[START_NODE[0]][START_NODE[1]] = GREY
    agent_grid[i][j] = RED
    START_NODE = (i,j)
    #update_frame()
set_start(START_NODE[0], START_NODE[1])

def set_end(i,j):
    global END_NODE
    actual_grid[END_NODE[0]][END_NODE[1]] = WHITE
    #display_grid[i][j] = BLUE
    agent_grid[END_NODE[0]][END_NODE[1]] = GREY
    agent_grid[i][j] = BLUE
    END_NODE = (i,j)
    #update_frame()
set_start(START_NODE[0], START_NODE[1])
set_end(END_NODE[0], END_NODE[1])
init_agent_grid()

def set_blocked(i,j):
    if ((i,j)!=START_NODE) & ((i,j)!=END_NODE):
        actual_grid[i][j] = BLACK
        #update_frame()

def set_unblocked(i,j):
    if ((i,j)!=START_NODE) & ((i,j)!=END_NODE):
        actual_grid[i][j] = WHITE
        #update_frame()

def makepath(camefrom, current, start):
    #print(camefrom)
    total_path = []
    while current!=start:
        total_path.append(current)
        current = camefrom[current]
    total_path.append(start)
    return total_path

def give_neighbors_agent(NODE):
    retval = []
    i,j = NODE[0], NODE[1]
    #print('Indices received:', i,j)
    if j-1>=0:
        if agent_grid[i][j-1]!=BLACK:
            retval.append((i, j-1))
    if j+1<ROWS:
        if agent_grid[i][j+1]!=BLACK:
            retval.append((i, j+1))
    if i-1>=0:
        if agent_grid[i-1][j]!=BLACK:
            retval.append((i-1, j))
    if i+1<ROWS:
        if agent_grid[i+1][j]!=BLACK:
            retval.append((i+1, j))
    return retval


h = [[0 for j in range(ROWS)] for i in range(ROWS)]


from CustomPriorityQueue_maxG import CustomPQ_maxG as CQ
def astar_agent_backward(start, counter):
    target = start
    start = END_NODE
    #print(target)
    camefrom = dict()
    openset = CQ()
    openset.put((h[start[0]][start[1]], start, g[start[0]][start[1]]))
    #print('AFTER PUT:', openset.queue)
    visited = set()


    #g[start[0]][start[1]] = 0
    #g[END_NODE[0]][END_NODE[1]] = float('inf')
    #search = [[0 for _ in range(ROWS)] for x in range(ROWS)]
    #f[start[0]][start[1]] = h[start[0]][start[1]]
    #print(openset)
    #print(openset.empty())
    while not openset.empty():
        choice = openset.get()
        if choice not in visited:
            visited.add(choice)
            #print('AFTER GET:', openset.queue)
            #print('Choice:', choice)
            choice = choice[1]
            if choice==target:
                path = makepath(camefrom, target, start)
                return True, path
            for neighbor in give_neighbors_agent(choice):
                if search[neighbor[0]][neighbor[1]] < counter:
                    g[neighbor[0]][neighbor[1]] = float('inf')
                    search[neighbor[0]][neighbor[1]] = counter
                test = g[choice[0]][choice[1]]+1
                if test<g[neighbor[0]][neighbor[1]]:
                    camefrom[neighbor] = choice
                    g[neighbor[0]][neighbor[1]] = test

                    f[neighbor[0]][neighbor[1]] = test+h[neighbor[0]][neighbor[1]]
                    if neighbor not in openset.queue:
                        openset.put((f[neighbor[0]][neighbor[1]], neighbor, g[neighbor[0]][neighbor[1]]))
                    #print('AFTER PUT:', openset.queue)
    return False, None


def astar_agent(start, counter):
    camefrom = dict()
    openset = CQ()
    openset.put((h[start[0]][start[1]], start, g[start[0]][start[1]]))
    visited = set()
    while not openset.empty():
        choice = openset.get()
        if choice not in visited:
            visited.add(choice)
            choice = choice[1]
            if choice==END_NODE:
                path = makepath(camefrom, END_NODE, start)
                return True, path[::-1]
            for neighbor in give_neighbors_agent(choice):
                if search[neighbor[0]][neighbor[1]] < counter:
                    g[neighbor[0]][neighbor[1]] = float('inf')
                    search[neighbor[0]][neighbor[1]] = counter
                test = g[choice[0]][choice[1]]+1
                if test<g[neighbor[0]][neighbor[1]]:
                    camefrom[neighbor] = choice
                    g[neighbor[0]][neighbor[1]] = test
                    f[neighbor[0]][neighbor[1]] = test+h[neighbor[0]][neighbor[1]]
                    if neighbor not in openset.queue:
                        openset.put((f[neighbor[0]][neighbor[1]], neighbor, g[neighbor[0]][neighbor[1]]))
    return False, None

def repeated_astar(start):
    Open_path = set()
    counter = 0
    calculate_hcost(ROWS, END_NODE)
    current_location = start
    agent_grid[current_location[0]][current_location[1]] = RED
    Open_path.add(start)

    while current_location!=END_NODE:
        counter+=1
        g[start[0]][start[1]] = 0
        search[start[0]][start[1]] = counter
        g[END_NODE[0]][END_NODE[1]] = float('inf')
        search[END_NODE[0]][END_NODE[1]] = counter
        ans, path = astar_agent(current_location, counter)
        i = 1
        if ans:
            next_step = path[1]
            steps_taken_without_path_update = 0
            while agent_grid[next_step[0]][next_step[1]]!=BLACK:
                steps_taken_without_path_update+=1
                i+=1
                agent_grid[current_location[0]][current_location[1]] = WHITE
                current_location = next_step #Move a step
                Open_path.add(current_location)
                agent_grid[current_location[0]][current_location[1]] = RED
                if current_location==END_NODE:
#                     print("Astar computed path", counter, 'times.')
                    return True, len(Open_path)
                #Update information about surroundings
                updates = give_neighbors_update(current_location)
                for node in updates:
                    if agent_grid[node[0]][node[1]]!=PATH:
                        agent_grid[node[0]][node[1]] = actual_grid[node[0]][node[1]]
                #update_frame()
                next_step = path[i] #Get ready to take next step
                #update knowledge
        
        #repeat the loop when a block is found
        else:
            return False, len(Open_path)
#     print("Astar computed path", counter, 'times.')
    return True, len(Open_path)
    pass

def astar_agent_adaptive(start, counter, gGOAL):
    #print(start)
    camefrom = dict()
    openset = CQ()
    openset.put((h[start[0]][start[1]], start, g[start[0]][start[1]]))
    visited = set()
    while not openset.empty():
        choice = openset.get()
        if choice not in visited:
            visited.add(choice)
            choice = choice[1]
            if choice==END_NODE:
                path = makepath(camefrom, END_NODE, start)
                return True, path[::-1], g[END_NODE[0]][END_NODE[1]]
            for neighbor in give_neighbors_agent(choice):
                if search[neighbor[0]][neighbor[1]] < counter:
                    g[neighbor[0]][neighbor[1]] = float('inf')
                    search[neighbor[0]][neighbor[1]] = counter
                test = g[choice[0]][choice[1]]+1
                if test<g[neighbor[0]][neighbor[1]]:
                    camefrom[neighbor] = choice
                    g[neighbor[0]][neighbor[1]] = test
                    f[neighbor[0]][neighbor[1]] = test+h[neighbor[0]][neighbor[1]]
                    h[neighbor[0]][neighbor[1]] = gGOAL - test

                    if neighbor not in openset.queue:
                        openset.put((f[neighbor[0]][neighbor[1]], neighbor, g[neighbor[0]][neighbor[1]]))

    return False, None, gGOAL

def repeated_astar_ADAPTIVE(start):
    counter = 0
    calculate_hcost(ROWS, END_NODE)
    current_location = start
    agent_grid[current_location[0]][current_location[1]] = RED
    g[start[0]][start[1]] = 0
    search[start[0]][start[1]] = counter
    g[END_NODE[0]][END_NODE[1]] - float('inf')
    search[END_NODE[0]][END_NODE[1]] = counter
    ans, path = astar_agent(current_location, counter)
    gGOAL = g[END_NODE[0]][END_NODE[1]]
    while current_location!=END_NODE:
        counter+=1
        #getpath from a star
        #print('---------------------------------')
        #print('CALCULATING PATH FROM:', current_location)
        h[start[0]][start[1]] = gGOAL - g[start[0]][start[1]]
        g[start[0]][start[1]] = 0
        search[start[0]][start[1]] = counter
        g[END_NODE[0]][END_NODE[1]] = float('inf')
        search[END_NODE[0]][END_NODE[1]] = counter
        ans, path, gGOAL = astar_agent_adaptive(current_location, counter, gGOAL)
        #print(path)
        #print('Path found:', ans)
        #print(path)
        #print('----------------------------------')
        #follow path until node in path is blocked
        #Assume that the first node in path would be start point and last would be end point
        i = 1
        if ans:
            next_step = path[1]
            while agent_grid[next_step[0]][next_step[1]]!=BLACK:
                i+=1
                #print('Currently at:', current_location)
                agent_grid[current_location[0]][current_location[1]] = WHITE
                #display_grid[current_location[0]][current_location[1]] = WHITE
                current_location = next_step #Move a step
                #display_grid[current_location[0]][current_location[1]] = RED
                #time.sleep(0.2)
                agent_grid[current_location[0]][current_location[1]] = RED
                if current_location==END_NODE:
                    return True
                #Update information about surroundings
                updates = give_neighbors_update(current_location)
                for node in updates:
                    if agent_grid[node[0]][node[1]]!=PATH:
                        agent_grid[node[0]][node[1]] = actual_grid[node[0]][node[1]]
                #update_frame()
                next_step = path[i] #Get ready to take next step
                #update knowledge
        
        #repeat the loop when a block is found
        else:
            return False
    print("Astar computed path", counter, 'times.')
    return True
    pass

def repeated_astar_backward(start):
    Open_path = set()
    counter = 0
    calculate_hcost(ROWS, END_NODE)
    current_location = start
    agent_grid[current_location[0]][current_location[1]] = RED
    Open_path.add(start)

    while current_location!=END_NODE:
        counter+=1
        #getpath from a star
        ##print('---------------------------------')
        ##print('CALCULATING PATH FROM:', current_location)
        g[END_NODE[0]][END_NODE[1]] = 0
        
        search[start[0]][start[1]] = counter
        g[start[0]][start[1]] = float('inf')
        search[END_NODE[0]][END_NODE[1]] = counter
        ans, path = astar_agent_backward(current_location, counter)
        #print(path)
        #print('Path found:', ans)
        #print(path)
        ##print('----------------------------------')
        #follow path until node in path is blocked
        #Assume that the first node in path would be start point and last would be end point
        i = 1
        if ans:
            next_step = path[1]
            steps_taken_without_path_update = 0
            while agent_grid[next_step[0]][next_step[1]]!=BLACK:
                steps_taken_without_path_update+=1
                i+=1
                #print('Currently at:', current_location)
                agent_grid[current_location[0]][current_location[1]] = WHITE
                #display_grid[current_location[0]][current_location[1]] = WHITE
                current_location = next_step #Move a step
                #display_grid[current_location[0]][current_location[1]] = RED
                #time.sleep(0.2)
                Open_path.add(current_location)
                agent_grid[current_location[0]][current_location[1]] = RED
                if current_location==END_NODE:
                    return True, len(Open_path)
                #Update information about surroundings
                updates = give_neighbors_update(current_location)
                for node in updates:
                    if agent_grid[node[0]][node[1]]!=PATH:
                        agent_grid[node[0]][node[1]] = actual_grid[node[0]][node[1]]
                #update_frame()
                next_step = path[i] #Get ready to take next step
                #update knowledge
        
        #repeat the loop when a block is found
        else:
            return False, len(Open_path)



    return True, len(Open_path)
    pass

            
def SE_PHASE(active):
    se_phase = True
    while se_phase&active:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                active = False
                break
             
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_presses = pygame.mouse.get_pressed()
                if mouse_presses[0]: #Tracks left click
                    position = pygame.mouse.get_pos()
                    i = (position[0]//NODE_LENGTH)
                    j = (position[1]//NODE_LENGTH)
                    if (i<ROWS)&(j<ROWS):
                        if not ((actual_grid[i][j]==BLACK)or(actual_grid[i][j]==BLUE)):
                            set_start(i,j)
                            init_agent_grid()

                if mouse_presses[2]: #Tracks Right click
                    position = pygame.mouse.get_pos()
                    i = (position[0]//NODE_LENGTH)
                    j = (position[1]//NODE_LENGTH)
                    if (i<ROWS)&(j<ROWS):
                        if not ((actual_grid[i][j]==BLACK)or(actual_grid[i][j]==RED)):
                            set_end(i,j)
            
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_s:
                    position = pygame.mouse.get_pos()
                    i = (position[0]//NODE_LENGTH)
                    j = (position[1]//NODE_LENGTH)
                    set_start(i,j)
                    #print("Start point has been changed to:", (i,j))

                if event.key == pygame.K_e:
                    position = pygame.mouse.get_pos()
                    i = (position[0]//NODE_LENGTH)
                    j = (position[1]//NODE_LENGTH)
                    set_end(i,j)
                    #print("End point has been changed to:", (i,j))

                if event.key == pygame.K_c:
                    se_phase = False
                    break
    return active

def BLOCK_PHASE(active):
    calculate_hcost(ROWS, END_NODE)
    g = [[float('inf') for _ in range(ROWS)] for x in range(ROWS)]
    f = [[float('inf') for _ in range(ROWS)] for x in range(ROWS)]
    block_phase = True
    while block_phase&active:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                active = False
                break
             
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_presses = pygame.mouse.get_pressed()
                if mouse_presses[0]: #Tracks left click
                    position = pygame.mouse.get_pos()
                    i = (position[0]//NODE_LENGTH)
                    j = (position[1]//NODE_LENGTH)
                    set_blocked(i,j)

                if mouse_presses[2]: #Tracks right click
                    position = pygame.mouse.get_pos()
                    i = (position[0]//NODE_LENGTH)
                    j = (position[1]//NODE_LENGTH)
                    set_unblocked(i,j)

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_c:
                    block_phase = False
                    break
    return active

def ASTAR_PHASE(active):
    astar_phase = True
    count = 0
    while astar_phase&active:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                active = False
                break
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_c:
                    astar_phase = False
                    break
        if count<1:
            #ans = repeated_astar_backward(START_NODE)
            ans = repeated_astar_ADAPTIVE(START_NODE)
            #ans = repeated_astar(START_NODE)
            if ans:
                print("REACHED TARGET")
            else:
                print("CANT REACH DUDE")
            count+=1
    return active

def game_loop():
    #update_frame()
    active = True
    active = SE_PHASE(active)
    active = BLOCK_PHASE(active)
    active = ASTAR_PHASE(active)

def stat_loop(i):
    global actual_grid
    global agent_grid
    global search
    global g
    global f
    global successful, unsuccessful
    set_start(START_NODE[0], START_NODE[1])
    set_end(END_NODE[0], END_NODE[1])
    init_agent_grid()

    h = [[0 for j in range(ROWS)] for i in range(ROWS)]
    calculate_hcost(ROWS, END_NODE)
    fname = 'attach2\Random_Maze '+str(i)
    actual_grid = readFile(fname)
    agent_grid = [[GREY for _ in range(ROWS)] for x in range(ROWS)]
    agent_grid[-1][-1] = BLUE
    search = [[0 for _ in range(ROWS)] for x in range(ROWS)]
    g = [[float('inf') for _ in range(ROWS)] for x in range(ROWS)]
    f = [[float('inf') for _ in range(ROWS)] for x in range(ROWS)]
    ans, num_of_expanded_cells = repeated_astar_backward(START_NODE)
#     ans = repeated_astar_ADAPTIVE(START_NODE)
#     ans, num_of_expanded_cells = repeated_astar(START_NODE)
#     expanded_cells.append(num_of_expanded_cells)
    if ans:
        print("REACHED TARGET")
    else:
        print("CANT REACH DUDE")
    return ans
    pass

def main():
    game_loop()
files = [2, 4, 6, 9, 12, 15, 17, 18, 20, 21, 22, 27, 28, 29, 30, 34, 39, 40, 44, 49, 50, 52, 54, 55, 61, 62, 65, 68, 69, 72, 77, 83, 85, 86, 88, 89, 97, 99, 1, 3, 5, 24, 8, 10, 11, 13, 26, 31, 32, 35]
# for i in files:
#     stat_loop(i)
# print('Expanded Cells: ', expanded_cells)
import time 
times = [time.time()]
for i in files:
    stat_loop(i)
    times.append(time.time()-times[-1])
print('Expanded Cells: ', times)
print('Respective Files', files)

REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
REACHED TARGET
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
CANT REACH DUDE
Expanded Cells:  [1665884744.5385177, 3.674888849258423, 1665884749.0394504, 8.508991003036499, 1665884751.839931, 13.866939544677734, 1665884756.1711195, 17.26581835746765, 1665884760.3224685, 19.983552932739258, 1665884764.0375335, 22.8

In [18]:
maxG_bwd = [3.674888849258423, 1665884749.0394504, 8.508991003036499, 1665884751.839931, 13.866939544677734, 1665884756.1711195, 17.26581835746765, 1665884760.3224685, 19.983552932739258, 1665884764.0375335, 22.851881980895996, 1665884768.2453125, 26.66764497756958, 1665884772.937763, 30.49366855621338, 1665884776.529512, 33.55267333984375, 1665884780.4878922, 36.89573264122009, 1665884783.7003012, 40.9459342956543, 1665884787.1510432, 46.40433216094971, 1665884791.4276075, 50.046456813812256, 1665884794.6250558, 53.634748697280884, 1665884798.9115913, 58.43491220474243, 1665884803.0964005, 62.82758378982544, 1665884808.2824113, 67.60247492790222, 1665884813.963188, 70.66631197929382, 1665884817.5606444, 73.10382580757141, 1665884822.3069186, 76.92859673500061, 1665884826.605453, 77.11223196983337, 1665884826.9665182, 80.69661474227905, 1665884832.218472, 84.09056997299194, 1665884836.3104978, 85.80823063850403, 1665884839.7602718, 86.09845495223999, 1665884844.0252287]

In [19]:
# maxG_fwd = [290, 393, 318, 320, 286, 296, 300, 388, 296, 332, 321, 295, 305, 392, 302, 326, 443, 357, 415, 326, 299, 296, 519, 293, 269, 347, 306, 355, 304, 316, 363, 306, 315, 316, 375, 342, 269, 334, 422, 310, 3, 5, 294, 355, 3, 528, 54, 338, 2, 308]
# random_maze_ids = [2, 4, 6, 9, 12, 15, 17, 18, 20, 21, 22, 27, 28, 29, 30, 34, 39, 40, 44, 49, 50, 52, 54, 55, 61, 62, 65, 68, 69, 72, 77, 83, 85, 86, 88, 89, 97, 99, 1, 3, 5, 24, 8, 10, 11, 13, 25, 16, 19, 23]
# maxG_bwd = expanded_cells
# for x,y,z in zip(random_maze_ids, maxG_fwd, maxG_bwd):
#     print("For Random_Maze:", x, 'expanded in favor of maxG_fwd:',y, 'vs maxG_bwd:', z)

In [16]:
print(sum(maxG_fwd))
print(sum(maxG_bwd))

15247
14277
